In [1]:
# Setup
import pattern.web as web
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from cs109style import customize_mpl, customize_css
customize_mpl()
customize_css()
%pylab inline

Setting custom matplotlib visual style
Setting custom CSS for the IPython Notebook
Populating the interactive namespace from numpy and matplotlib


C:\Users\rlochavillo\Anaconda2\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## Example 2: extracting reddit titles, upvotes, downvotes, and submission time

### We'll operate in two phases:
* first, find all the URLs to comment pages on the first few front pages of reddit.
* second, extract information from each comments page

In [4]:
url = web.URL('http://www.reddit.com/')
dom = url.download(cached=False)


In [13]:
def get_links_from_front_pages(n, dom):
    'find  URLs of comments pages, linked from the n first few pages of reddit'
    #url = web.URL('http://www.reddit.com/')
    comment_pages = []
    for page_idx in range(n):
        dom = web.DOM(dom)
        links = [li.by_tag("a") for li in dom.by_tag("li.first")]
        comment_pages.append([link[0].attributes.get('href','') for link in links]) #append in ascii, not as list but to the end
        ### Extract comments pages
        
        ### find the next page link - reddit has 25 links per page

    # use set() to remove repeated URLs
    print comment_pages
    return comment_pages

            
print len(get_links_from_front_pages(1, dom))

[[u'https://www.reddit.com/r/pics/comments/4bbvve/nice_save/', u'https://www.reddit.com/r/Showerthoughts/comments/4bbvlz/it_took_me_5_minutes_to_smoke_a_cigarette_which/', u'https://www.reddit.com/r/videos/comments/4bbvxm/i_cant_believe_i_watched_this_the_most_annoying/', u'https://www.reddit.com/r/mildlyinteresting/comments/4bbwbk/my_chemistry_teacher_posted_this_titled_dont/', u'https://www.reddit.com/r/GetMotivated/comments/4bbw2s/discussion/', u'https://www.reddit.com/r/Jokes/comments/4bbw8s/whos_the_president_with_the_dirtiest_hands/', u'https://www.reddit.com/r/gifs/comments/4bbvlu/always_stay_on_your_lane/', u'https://www.reddit.com/r/Fitness/comments/4bbvvc/lifting_shoes_in_australia/', u'https://www.reddit.com/r/sports/comments/4bbkls/novak_djokovic_questions_equal_prize_money_in/', u'https://www.reddit.com/r/UpliftingNews/comments/4bbvp5/stephen_colbert_funds_1000_projects_for_south/', u'https://www.reddit.com/r/pics/comments/4bar8e/my_boyfriend_refused_to_believe_i_had_met_t

In [ ]:
def info_from_comments_pages(links):
    'fetch title, upvotes, downvotes, time of submission from a sequence of links'
    results = []
    for urltext in links:
        url = web.URL(urltext)
        print "fetching info for", url
        try:
            dom = web.DOM(url.download(cached=False))
            
            ### Extract title, upvotes, downvotes, submission time
            
            results.append((title, upvotes, downvotes, pd.to_datetime(time)))
        except KeyboardInterrupt:
            # allow us to interrupt the kernel and still continue
            break
        except:
            pass  # some things that look like comment pages don't have the information above
    return results

In [ ]:
comments_pages = get_links_from_front_pages(5)
print "Fetching info for", len(comments_pages), "pages"
pages = info_from_comments_pages(comments_pages)
titles, upvotes, downvotes, dates = zip(*pages)  # zip(*seq) transposes a sequence of sequences.
df = pd.DataFrame({'title' : titles, 'upvotes' : upvotes, 'downvotes' : downvotes, 'date' : dates}, index=dates)
print df

In [ ]:
df.sort('date', inplace=True)
df['upvotes'].plot(c='g')
df['downvotes'].plot(c='r')
(df['upvotes'] - df['downvotes']).plot(c='k')
